### <$v_\pi$를 추정하기 위한 표 형태로 된 TD(0)>

입력 : 평가 대상이 될 정책 $\pi$

알고리즘 파라미터 : 시간 간격 $\alpha \in (0,1] $ 

모든 $s \in S^+$ 에 대한 V(s)를 임의의 값으로 초기화. 단, V(종단) = 0 

각 에피소드에 대한 루프 : 
- S를 초기화 
- 에피소드의 각 단계에 대한 루프 : 

> A ← S에 대해 $\pi$에 따라 도출된 행동 

> 행동 A를 취하고 $R,S'$ 을 관측 

> V(s) ← $V(S) + \alpha[R+ \gamma V(S') - V(S)] $

> S ← S' 

S가 종단이면 종료 

**<구현해야 하는 것>**
- V(S) : 상태 s에 대한 가치 함수 계산. 챕터 4의 코드 참고 
- next_S(s,a) : 상태 s와 행동 a를 했을 때의 s'. 
- make_episode : 챕터 4의 코드 참고

**<필요한 것>** 
- $\alpha$ : class 제작시 입력 값으로 부여 


**<함수 / 데이터 형식>** 
- class evaluate_TD : # alpha 값 추가 
> def __init__(self, S, A, pi, alpha reward_func, epsilon = 0.001, gamma = 0.9, num_episode = 10, len_episode = 20) : 


**<외부함수>** 
- $\pi(s,a) $ : 상태 s 하에서 선택할 a. 확률론적으로 구현할 것  
- R(s',a,s) : 보상함수. 챕터 4의 코드 참고 


**<고민점>** 
- V를 어떻게 임의의 값으로 초기화할 것인까? 단순히 0 값으로 부여하는 것보다 더 좋은 방법이 있나?



In [1]:
# 테스트 용 임시 데이터 
S = list(range(100)) 
A = list(range(-5,5))

In [2]:
from collections import defaultdict
import random
import numpy as np

In [3]:
# 외부용 함수 reward_func 간략 구현 (이전 예시 활용)
def reward_func(next_s, a, s) : 
    # next_s 와 s의 차이가 짝수이면 +1, 홀수면 -1 
    # 단, a의 크기에 반비례함. 
    if abs(next_s - s) %2 == 0 : reward = 1 
    else : reward = -1
    
    if a == 0 : 
        return 0 
    else : 
        return reward / a # 즉, a가 양수이며 짝수이며, 가능한 작을 때 (=2) 일 때 최대의 보상이 주어지도록 설정 
    
    
def pi(s,a) : #s 상황에서 a를 선택할 확률. 전체 합은 1이여야 한다. 
    # 확률은 모두 동일하게 설정 
    
    return 1/10

In [50]:
# 비활성 MC 정책 코드 참고
# 기존 코드를 전반적으로 일반화시킴

class evaluate_TD :  
    def __init__(self, S, A, pi, reward_func, alpha=0.1,  epsilon = 0.001, gamma = 0.9, num_episode = 10, len_episode = 20) : 
        self.S = S 
        self.A = A 
        self.pi = pi 
        self.reward_func = reward_func
        self.alpha = alpha
        self.epsilon = epsilon 
        self.gamma = gamma 
        self.num_episode = num_episode
        self.T = len_episode

#        self.b, self.C = self.initiate_b()
#        self.Q, self.return_lst, self.s_prob = self.initiate() 
        self.V = self.initiate_V() 
    
    def initiate_V(self) : # V 값을 초기화 
        return [0]*len(self.S)
    

    def choice_action(self, s, policy) : #수정 - 일반화. 정책 기반으로 상황 s에 있을 때 선택할 행동 a 산출 
        policy_a_list = [] 
        for _ in self.A :
            policy_a_list.append(policy(s,_)) 

        a= random.choices(self.A, weights = policy_a_list)
        a = a[0]
        return a
    
    def next_s(self, s,a) : # 상태 s에서 a 행동을 했을 때 다음 상태 s'. 정책, S,A 에 따라 달라짐. 
        return min(max(s+a, 0), max(self.S)) 


    def make_episode(self, start_s, T) :
        s = start_s
        episode = {"S" : [], "A" : [], "R" : []}
        episode["R"].append(0) # R_0 값 부여 
        for _ in range(T) : 
            episode["S"].append(s)
            a = self.choice_action(s, self.pi) 
            next_s = self.next_s(s, a)
            r = self.reward_func(next_s, a, s)
            episode["A"].append(a)
            episode["R"].append(r)
            s = next_s
        return episode 
 
    def update_returns(self) : # TD(0) 형식에 맞춰 대거 수정 
        
        for _ in range(self.num_episode) : 
            start_s = random.choice(self.S) # 시작 탐험 가정 
            episode = self.make_episode(start_s, self.T) 
            S,R,A = episode['S'],episode['R'], episode['A'] 
            # make_episode 에서 이미 a,r를 계산해 두었기 때문에 V(s)만 갱신하겠음. 
            for index, s in enumerate(S[:-1]) :  
                index_origin_s = self.S.index(s)
                index_origin_next_s = self.S.index(S[index+1])
                self.V[index_origin_s] = self.V[index_origin_s] + self.alpha*(R[index+1] +self.gamma * self.V[index_origin_next_s] - self.V[index_origin_s] )    
                
        

In [55]:
test = evaluate_TD(S,A, pi, reward_func)
lst = [] 

for _ in range(50) : 
    lst.append(test.V[:])
    test.update_returns()

print(lst[0])
print(lst[-1])



# Q. 왜 업데이트를 한번 한 것과, 여러번 한 것의 결과가 같지? 이미 최적의 가치 함수라서 그런건가? 
# 값 할당 실수가 있었음. 

"""
a = test.V
test.update_returns()
b = test.V
test.update_returns() 
c = test.V

위의 코드의 경우 a,b,c 모두에 test.V 라는 동일한 객체를 할당한 것임. 
따라서 최초에 a의 값이 다른 것이었다 하더라도, test.V를 업데이트 하면 a,b,c 모든 값이 바뀌는 것임. 

즉, V의 값이 변하는 것을 따로 저장해두고 싶으면 test.V[:] 을 통해 값을 "복사" 해야한다. 

"""

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[-1.0493505055941106, -0.43038595743146524, -0.4169946350929829, -0.5877144751062008, -0.25694432747784957, -0.4720657832093836, -0.2639135784257529, -0.3220448175060952, -0.03826029381856259, -0.22811732853311537, 0.07114026421328888, -0.031585929277619405, 0.12330239095191457, 0.03155697016806558, -0.04166150204111681, 0.06276496680687738, 0.1576356438373627, 0.00908920916935058, 0.08607635605511552, 0.1787902954937421, 0.31653495890584277, 0.008272978683048843, 0.269545528176297, 0.006074211442367562, 0.019513235378787158, -0.0020004820785587962, -0.09530570428342128, -0.055898457718203676, 0.01328969478283342, -0.06999968850104638, 0.2569413505085248, 0.022524517793944633, 0.13145100505

'\na = test.V\ntest.update_returns()\nb = test.V\ntest.update_returns() \nc = test.V\n\n위의 코드의 경우 a,b,c 모두에 test.V 라는 동일한 객체를 할당한 것임. \n따라서 최초에 a의 값이 다른 것이었다 하더라도, test.V를 업데이트 하면 a,b,c 모든 값이 바뀌는 것임. \n\n즉, V의 값이 변하는 것을 따로 저장해두고 싶으면 test.V[:] 을 통해 값을 "복사" 해야한다. \n\n'

In [24]:
a=[1,2,3,4,5,6,7]

print(a[:-1])

[1, 2, 3, 4, 5, 6]
